# TSS, RSS, ESS

### Решение задачи 1.1.1 из [Листка 03](https://nbviewer.org/github/artamonoff/Econometrica/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B8/List03-Goodness-of-Fit.html)

In [1]:
import numpy as np
import pandas as pd
import io
import statsmodels.formula.api as smf

In [2]:
# определяем данные
data_string ='''log(output) log(capital) log(labour) y.hat
2.22            0.96        5.21     2.19 
1.30            0.28        4.51     1.47 
3.36            3.10        6.05     3.23 
1.42            2.37        4.28     1.56 
1.06            0.15        3.83     0.85 
6.17            5.62        8.94     6.15 
0.77            0.52        3.87     0.94 
1.42            0.15        4.68     1.61 
0.90           -0.13        3.56     0.57 
1.35            1.25        4.28     1.40 '''

In [3]:
# создаём DataFrame
df = pd.read_csv(io.StringIO(data_string), sep='\s+')
df

,log(output),log(capital),log(labour),y.hat
0,2.22,0.96,5.21,2.19
1,1.30,0.28,4.51,1.47
2,3.36,3.10,6.05,3.23
3,1.42,2.37,4.28,1.56
4,1.06,0.15,3.83,0.85
5,6.17,5.62,8.94,6.15
6,0.77,0.52,3.87,0.94
7,1.42,0.15,4.68,1.61
8,0.90,-0.13,3.56,0.57
9,1.35,1.25,4.28,1.40


In [4]:
# Переименуем столбцы, так как log(output) интерпреитуется как функция логарифма от outpu
df =df.rename(columns={"log(output)": "LOGoutput", "log(capital)": "LOGcapital", "log(labour)":'LOGlabour'})
df

,LOGoutput,LOGcapital,LOGlabour,y.hat
0,2.22,0.96,5.21,2.19
1,1.30,0.28,4.51,1.47
2,3.36,3.10,6.05,3.23
3,1.42,2.37,4.28,1.56
4,1.06,0.15,3.83,0.85
5,6.17,5.62,8.94,6.15
6,0.77,0.52,3.87,0.94
7,1.42,0.15,4.68,1.61
8,0.90,-0.13,3.56,0.57
9,1.35,1.25,4.28,1.40


#### Первый способ вычисления парамметров регрессии. Согласно определению

In [5]:
# Вычислим общую сумму квадратов
TSS = ((df['LOGoutput'] - np.mean(df['LOGoutput']))**2).sum()
TSS.round(3)

24.479

In [6]:
# Вычислим объяснённую сумму квадратов
ESS = ((df['y.hat']- np.mean(df['LOGoutput']))**2).sum()

In [7]:
# Вычисли остаточную сумму квадртов
RSS = ((df['LOGoutput'] - df['y.hat'])**2).sum()
RSS.round(3)

0.287

In [8]:
# Коэффициент детерминации равен
R2 = ESS.round(3)/TSS.round(3)
R2.round(3)

0.991

In [9]:
# Вычислим исправленный коэффициенте детерминации
n = len(df) # количество измерений равно длине строк в исходной таблице минус 1, т.к. присутствует строка заголовака 
k = 2 # количество регрессорово
RSS_adj = 1 - (RSS/TSS)*((n-1)/(n-k))
RSS_adj

0.9868007211193773

#### Второй способ вычисления парамметров регрессии

In [10]:
# Построем регрессионную модель. Обратите внимание, что так как в таблице даны значения логарифмом, то функция
# np.log() мы не применяем
fitted_line = smf.ols(formula='LOGoutput~LOGcapital+LOGlabour', data=df).fit()

In [11]:
# Предсказанные значения регрессии 
fitted_line.predict().round(2)

array([2.19, 1.47, 3.23, 1.56, 0.85, 6.14, 0.94, 1.61, 0.57, 1.4 ])

In [12]:
# Предсказанные значения регрессии в исходной таблице
np.array(df['y.hat'])

array([2.19, 1.47, 3.23, 1.56, 0.85, 6.15, 0.94, 1.61, 0.57, 1.4 ])

In [13]:
# Параметры регрессии
fitted_line.params

Intercept    -2.568462
LOGcapital    0.138515
LOGlabour     0.887584
dtype: float64

In [14]:
# Вычислим объяснённую сумму квадратов, используя регрессионную модель
fitted_line.ess.round(3)

24.195

In [15]:
# Вычислим полную сумму квадратов, используя регрессионную модель
fitted_line.centered_tss.round(3)

24.479

In [16]:
# Вычислим остаточную сумму квадратов, используя регрессионную модель
fitted_line.ssr.round(3)

0.284

In [17]:
# Вычислим коэффициент детерминации, используя регрессионную модель
fitted_line.rsquared.round(3)

0.988

In [18]:
fitted_line.rsquared_adj

0.9851056175825208